In [ ]:
#imports and preprocessing, setting up train_loader


In [1]:
import matplotlib.pyplot as plt
from torchvision import transforms
from dataset import utils
from dataset.cub2011 import Cub2011
from dataset.CombinedDataLoader import CDL

D:\Programy\DataSpell 2022.2.3\DataspellProjects\SimCLRPytorch-main\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'cv2'

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
    gpu_index = 0
else:
    device = torch.device('cpu')
    gpu_index = -1


In [ ]:
root = './dataset/data'

In [ ]:
ds_whole = Cub2011(root, train=None, download=True, transform=transforms.ToTensor())

In [ ]:
utils.crop_all_images(root)

In [ ]:
aug_gray = transforms.Compose([transforms.Grayscale(), transforms.ToTensor()])
name_gray = 'grayscale'
utils.augmentation_on_all_images(root, aug_gray, name_gray)

In [ ]:
aug_gaus = transforms.Compose([transforms.GaussianBlur(kernel_size=5), transforms.ToTensor()])
name_gaus = 'gaussianBlur'
utils.augmentation_on_all_images(root, aug_gaus, name_gaus)

In [ ]:
aug_jitt = transforms.Compose([transforms.ColorJitter(brightness=.5, saturation=0.1, hue=.3), transforms.ToTensor()])
name_jitt = 'colorJitter'
utils.augmentation_on_all_images(root, aug_jitt, name_jitt)

In [ ]:
train_loader = CDL(root, train=None, transform=[transforms.resize(224) ,transforms.ToTensor()], num_classes=3, images=['cropped', name_gray, name_gaus, name_jitt])

In [ ]:
# declare train_dateset and train_loader

In [ ]:
from models.classificationModel import ClassificationModel
from models.classificationTrainer import Trainer

model = ClassificationModel(out_dim=20, dataset='')



In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                       last_epoch=-1)

In [ ]:
with torch.cuda.device(gpu_index):
    trainer = Trainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device, epochs=200, log_every_n_steps=100,
                      log_dir='test')
    trainer.train(train_loader)

In [ ]:
torch.save(model.state_dict(), 'saved_models/test.pth')